## Configs

In [1]:
offset = 0
start_date = '2012-06-08'
end_date = '2023-04-21'
# limit = 3000
# period = 'max' # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

## Download all NASDAQ traded symbols

In [ ]:
import pandas as pd

data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data = data[data['Test Issue'] == 'N']
data_clean = data[data['NASDAQ Symbol'].str.fullmatch(r'[A-Z]{1,5}', na = False)]
symbols = data_clean['NASDAQ Symbol'].tolist()
print('total number of symbols traded = {}'.format(len(symbols)))

total number of symbols traded = 11192


## Download Historic data

In [3]:
# ! pip install yfinance > /dev/null 2>&1
! mkdir hist

In [4]:
import yfinance as yf
import os, contextlib

In [5]:
%%time

limit = len(symbols)
end = min(offset + limit, len(symbols))
is_valid = [False] * len(symbols)
# force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in range(offset, end):
            s = symbols[i]
            data = yf.download(s,
                               start = start_date,
                               end = end_date,
                               auto_adjust = False,
                               progress = False)
            if len(data.index) == 0:
                continue
        
            is_valid[i] = True
            data.to_csv('hist/{}.csv'.format(s))

print('Total number of valid symbols downloaded = {}'.format(sum(is_valid)))


1 Failed download:
['AAAA']: YFPricesMissingError('possibly delisted; no price data found  (1d 2012-06-08 -> 2023-04-21) (Yahoo error = "Data doesn\'t exist for startDate = 1339128000, endDate = 1682049600")')

1 Failed download:
['AACB']: YFPricesMissingError('possibly delisted; no price data found  (1d 2012-06-08 -> 2023-04-21) (Yahoo error = "Data doesn\'t exist for startDate = 1339128000, endDate = 1682049600")')

1 Failed download:
['AACBR']: YFPricesMissingError('possibly delisted; no price data found  (1d 2012-06-08 -> 2023-04-21)')

1 Failed download:
['AACBU']: YFPricesMissingError('possibly delisted; no price data found  (1d 2012-06-08 -> 2023-04-21) (Yahoo error = "Data doesn\'t exist for startDate = 1339128000, endDate = 1682049600")')

1 Failed download:
['AACI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2012-06-08 -> 2023-04-21) (Yahoo error = "Data doesn\'t exist for startDate = 1339128000, endDate = 1682049600")')

1 Failed download:
['AACIU']:

Total number of valid symbols downloaded = 8145
CPU times: total: 8min 31s
Wall time: 1h 15min 54s


In [6]:
valid_data = data_clean[is_valid]
valid_data.to_csv('symbols_valid_meta.csv', index=False)

## Separating ETFs and Stocks

In [7]:
!mkdir stocks
!mkdir etfs

In [8]:
etfs = valid_data[valid_data['ETF'] == 'Y']['NASDAQ Symbol'].tolist()
stocks = valid_data[valid_data['ETF'] == 'N']['NASDAQ Symbol'].tolist()

In [9]:
import shutil
from os.path import isfile, join

def move_symbols(symbols, dest):
    for s in symbols:
        filename = '{}.csv'.format(s)
        shutil.move(join('hist', filename), join(dest, filename))
        
move_symbols(etfs, "etfs")
move_symbols(stocks, "stocks")

In [10]:
! rmdir hist